<a href="https://breakingbad.echosystem.fr/">![Breaking DDES](dados/titulo.png)</a>




*v3M sEr hAcKUdÃ0 4m19uiNh0*

# 1. DES

* *Data Encryption Standard* (IBM - 1975)
* Cifra de blocos (Rede Feistel - 16 rodadas)
* Tamanho do bloco: 64 bits
* Tamanho da chave: 56 bits (72,057,594,037,927,936 chaves)
    * Chave quebrada em 1999 ([EFF DES Cracker](https://en.wikipedia.org/wiki/EFF_DES_cracker),Força bruta ~22h)
* Algoritmo lento

### Cifra de blocos
Uma cifra de blocos consiste em dois algoritmos, um para criptografar *E* e outro para descriptografar *D*. Ambos os algoritmos aceitam dois parâmetros: Um bloco de comprimento conhecido e uma chave. O resultado do algoritmo é outro bloco.

A chave utilizada nas cifras de blocos é simétrica, ou seja, a mesma chave que faz a criptografia os dados também faz a descriptografia.

O algoritmo *E* recebe como parâmetros um bloco de **texto** (*plaintext*) e a chave, retornando um bloco de **texto cifrado** (*ciphertext*). Já a função de descriptografar *D* faz a operação inversa.

![Cifra de Blocos](dados/DES-block.png)

> ### Cifra de blocos cheatsheet
> * Opera em blocos de tamanho conhecido
> * Usa chaves simétricas
> * [AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) é provavelmente a cifra de blocos mais utilizada no mundo.

### Rede de Feistel

[Computerphile](https://www.youtube.com/watch?v=FGhj3CGxl8I&ab_channel=Computerphile)

Para os curiosos, uma rede de Feistel, ou cifra de bloco Luby–Rackoff, é o algoritmo base do DES. O funcionamento está além do escopo desse `notebook`, então fica aí a ilustração.

<img src="dados/Feistel_cipher_diagram_en.svg" style="width:50%;"></img>

Um fato interessante da rede de Feistel é que a função de descriptografar *D* é a mesma de criptografar *E*, porém com as sub-chaves invertidas.

Também notem o uso do operador *XOR* ⊕ na cifra, esse operador é muito utilizado em criptografia.

## Exemplinho de DES

In [1]:
from Crypto.Cipher import DES

chave = b'1OCTANOL'
texto = b"UMA METILAMINA POR FAVOR"

des = DES.new(chave, DES.MODE_ECB)

print("Texto original: ", texto)

cifrado = des.encrypt(texto)
print("Texto cifrado:  ", cifrado)

decifrado = des.decrypt(cifrado)
print("Texto decifrado:", decifrado)

Texto original:  b'UMA METILAMINA POR FAVOR'
Texto cifrado:   b'\xd9\x81G\xd8\xad\xd9\x88\xbeX\x9fy\xc02\x9a\x99\xff)0o%\xc5M\x03\xa8'
Texto decifrado: b'UMA METILAMINA POR FAVOR'


Algumas moléculas, nada a ver com o assunto.

<table>
    <tr>
        <td>
          <img src="dados/octanol.png"></img>
        </td>
        <td>
          <img src="dados/metilamina.png"></img>
        </td>
        <td>
          <img src="dados/sulfurico.png"></img>
        </td>
     </tr>
     <tr>
        <td style="text-align: center;">
            <a href="https://en.wikipedia.org/wiki/1-Octanol">Octanol</a>
        </td>
        <td style="text-align: center;">
            <a href="https://en.wikipedia.org/wiki/Methylamine">Metilamina</a>
        </td>
        <td style="text-align: center;">
            <a href="https://en.wikipedia.org/wiki/Sulfuric_acid">Ácido Sulfúrico</a>
        </td>
     </tr>
</table>

# 2. Breaking DES

Vamos tentar descriptografar os seguintes dados:
```python
cifra = b"\x8a\xa8!hpzjU&\xc6\x07#Z\x8d\xa9\xdd\xbf_=\x0fe\x8eV\xfa\xb6W\xf61;8\r\x8el\xd2\xab\x8eD\xc2\xc0\xac\x8cM\xe6\x8eK\x04#z[\x84q\x81\xfbc\xd3\xd7"
```

Podemos criar uma chave aleatória e ver no que dá:
```python
chave = b"sera?"
```

In [20]:
def pad(data, mod=8):
    """
    Garante que `data` sempre tem comprimento multiplo de `mod`.
    """
    padding = b" "*(mod - (len(data) % mod))
    padding = padding if len(padding) % mod != 0 else b""
    return data + padding

cifra = b'\x8a\xa8!hpzjU&\xc6\x07#Z\x8d\xa9\xdd\xbf_=\x0fe\x8eV\xfa\xb6W\xf61;8\r\x8el\xd2\xab\x8eD\xc2\xc0\xac\x8cM\xe6\x8eK\x04#z[\x84q\x81\xfbc\xd3\xd7'
chave = pad(b"sera?")
des = DES.new(chave, DES.MODE_ECB)
des.decrypt(cifra)

b"\x8e~\xaf'\x91\xcc&\xd4\xe1\xb6l\x1d\x93&\xc5\xf0&\xb7c\x0fl\xa5a\xc8K7\xf5\xef?O\xdd\xc4\xcc\xfat\x1e\xb1\x873\x9f\xb9\x8a\x84jf\xfaL!\xb3\xf9\xca\x10X\xb6\x91e"

Bom, o resultado não está nem perto de alguma mensagem inteligivel. Muito provavelmente não é essa a chave.

Para quebrar o DES, vamos nos aproveitar do fato que o espaço das chaves é relativamente pequeno (72 quadrilhões) e tentar cada chave possível.

...mentira, vamos reduzir ainda mais o espaço das chaves pra poder rodar em tempo hábil. Vamos considerar que a chave contém somente letras maiusculas e números. Também vamos considerar que a chave tem no máximo 5 caracteres. Isso vai reduzir o espaço das chaves pra perto de 60 milhões (36^5).

In [3]:
from itertools import product

def gerar_chave(alfabeto):
    MAXIMO = 5
    for i in range(MAXIMO+1):
        for c in product(alfabeto, repeat=i):
            yield "".join(c).encode("ascii")

alfabeto = "HABCDEFGIJKLMONPQSRTUVWXYZ0123456789"
chaves = gerar_chave(alfabeto)

# Mostra as chaves
regiao = 123456
for chave in [next(chaves) for _ in range(regiao+5)][regiao:]:
    des = DES.new(pad(chave), DES.MODE_ECB)
    print("Chave:", chave)
    print("Descriptografado:", des.decrypt(cifra))
    print()

Chave: b'AWIL'
Descriptografado: b'}\x01E\x97\x95#\xe6\xcb\xc5{\xc7\xa1\\\n\x8d\xd8!k\xbeb\xf6W\xdd\x95\x89R)Yv\xe2\x9b\x8dhGy\xdf\x10\xfaJ\xb9\xda_\xae\xe3o\xe1\x83M\xb4\xa8\xe88\x11z\x8b\xdb'

Chave: b'AWIM'
Descriptografado: b'}\x01E\x97\x95#\xe6\xcb\xc5{\xc7\xa1\\\n\x8d\xd8!k\xbeb\xf6W\xdd\x95\x89R)Yv\xe2\x9b\x8dhGy\xdf\x10\xfaJ\xb9\xda_\xae\xe3o\xe1\x83M\xb4\xa8\xe88\x11z\x8b\xdb'

Chave: b'AWIO'
Descriptografado: b'\x98\\\xe1\x04\xe9hd\xf5k\x85\x06\xafp.\xb7\xf3\xe6D\x13\xda\x98{\x9e\x94\xc9@\x84J\x9f\x87\xd7\x825\xed\xaf*\xe2\x93\xe1\xde\xb3|v\xcd\xef\xc8\xeb\xe0\x05\xb3\n\xd4\xaa\xc4]\x94'

Chave: b'AWIN'
Descriptografado: b'\x98\\\xe1\x04\xe9hd\xf5k\x85\x06\xafp.\xb7\xf3\xe6D\x13\xda\x98{\x9e\x94\xc9@\x84J\x9f\x87\xd7\x825\xed\xaf*\xe2\x93\xe1\xde\xb3|v\xcd\xef\xc8\xeb\xe0\x05\xb3\n\xd4\xaa\xc4]\x94'

Chave: b'AWIP'
Descriptografado: b'_\xd0\x96\x1cl\xf5\xe6\x11\xd3\x0b\xac\xda\x1a\xfc\x9a\xc4\x98\x9c:\xec:6W\x1c\x87-_\x00\xd1]mwnN\xb0+\xe5*\x94\x97\xae\x1a\x0b\xc5\xee[F\x8b\x

Vamos então tentar todas as chaves até encontrar o texto.

In [4]:
from tqdm.notebook import tqdm

def breaking_des():
    for chave in tqdm(gerar_chave(alfabeto), total=len(alfabeto) ** 5, desc="Breaking DES"):
        des = DES.new(pad(chave), DES.MODE_ECB)
        texto = des.decrypt(cifra)
        try:
            texto = texto.decode("ascii")
            print("ACHOOOOOU!")
            print("Texto:", texto)
            print("Chave:", chave)
            break
        except UnicodeDecodeError:
            pass

breaking_des()

Breaking DES:   0%|          | 0/60466176 [00:00<?, ?it/s]

ACHOOOOOU!
Texto: So you do have a plan! Yeah Mr. White! Yeah Science!    
Chave: b'H2SO4'


.

.

.

.

.

.

.

.

### UOOOU! Um minuto!

![yeah science](dados/yeah_science.gif)

É claro que estamos reduzindo muito o espaço de busca, mas acredite que é uma estratégia viável.
Em 1999 foi criada uma máquina ([EFF DES Cracker](https://en.wikipedia.org/wiki/EFF_DES_cracker)) capaz de tentar todas as chaves possíveis em aproximadamente 9 dias.

![DES Cracker](dados/Paul_kocher_deepcrack.jpg)

Hoje em dia é possível comprar hardware capaz de testar todas as combinações em 26 horas: https://crack.sh/

# 2. Double DES
Bom, se usar o DES uma vez não resiste ataques de força bruta, então vamos criptografar mais vezes o texto. Se rodarmos duas vezes o DES com chaves diferentes certamente teremos algo mais seguro.

In [22]:
_des_cache = {}
def des_cache(chave):
    if chave in _des_cache:
        return _des_cache[chave]
    
def double_des_enc(texto, chave1, chave2):
    des1 = DES.new(pad(chave1), DES.MODE_ECB)
    des2 = DES.new(pad(chave2), DES.MODE_ECB)
    return des2.encrypt(des1.encrypt(texto))

def double_des_dec(texto, chave1, chave2):
    des1 = DES.new(pad(chave1), DES.MODE_ECB)
    des2 = DES.new(pad(chave2), DES.MODE_ECB)
    return des1.decrypt(des2.decrypt(texto))

chave1 = b"123"
chave2 = b"456"
dcipher = double_des_enc(pad(b"choque"), chave1, chave2)
dtext = double_des_dec(dcipher, chave1, chave2)
dtext

b'choque  '


Certamente aplicando o DES duas vezes no texto vai ser seguro CERTO?

.

.

.

.

.

.

.

.

.

![errado](dados/achou-errado-otario.jpg)